# Steps to Preprocess the CORDEX-Australasia Ensemble for Input into Climpact
The code below includes the relevant preprocessing steps taken to prepare the daily tasmin, tasmax, and precip CORDEX-Australasia data for input into Climpact. <br>
We use 'preprocessing' here to describe steps taken to process the CORDEX data ahead of using the Climpact software to calculate the indices. <br>
Note, the following steps are taken to prepare the daily files:
- Merge the daily files into one file (stored as 1 file per year)
- Interpolate the files to a consistent 0.5 X 0.5 deg, rectilinear grid (noted as AUS-44i in the file naming)
- Create a new catalogue with the necessary metadata to use with the cordex-climpact software (see below)
- Additional troubleshooting where needed
- Sorting indices into directories based on the experiment and indice name

Using Climpact, we then calculate the full set of Climpact climate indices at a common 0.5 x 0.5 degree spatial resolution for the historical, RCP4.5, and RCP8.5 historical simulations. More information about climpact can be found at: https://climpact-sci.org/. A 'full set' of climate indices describes the 51 standardized indices calculated through Climpact. This is not exhaustive of all possible Climate Indices in existence. 

The revised Climpact software used to batch create the CORDEX-Australasia indices can be found at: https://github.com/coecms/cordex-climpact

Note, these preprocessing steps have already been done for the daily, historical and RCP8.5 precipitation datasets. <br>
Authors: Rachael N. Isphording, Josh Amoils, and (Alex) Ying Lung Liu

In [3]:
# Import Modules
import os
import fnmatch
import pandas as pd
import numpy as np
import xarray as xr

## Step 1: Find the files that we want and concatenate daily files into one dataset
Note, this has already been done for the historical and RCP8.5 precip, but needs to be done for the RCP4.5 simulations

In [1]:
# List regions (i.e. domains) that we are interested in
domains = ["AUS-22/", "AUS-44/", "AUS-44i/"]

# Define master paths (below are the two master paths on Gadi for the original CORDEX output)
master_path = '/g/data/al33/replicas/cordex/output/'
#master_path = '/g/data/rr3/publications/CORDEX/output/'

output_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step1_concat/'

In [ ]:
# Define useful functions

# Get dates from the first and last file to add to the new, concatenated (merged) file name
def extract_dates_from_filenames(filenames):
    start_date = None
    end_date = None

    for filename in filenames:
        date_range = filename.split('_')[-1].split('.')[0]
        range_start, range_end = date_range.split('-')

        if start_date is None or range_start < start_date:
            start_date = range_start
        if end_date is None or range_end > end_date:
            end_date = range_end

    return start_date, end_date

# Merge files using CDO
def merge_files(input_file, output_file):
    
    print("!START:\n input files: " + input_file + "\n output file: " + output_file)
    
    !cdo mergetime {input_file} {output_file}

    return(print("!COMPLETED:\n input files: " + input_file + "\n output file: " + output_file))

# Recursive function to parse directories and process files
def parse_directory(directory, domain):
    for root, dirs, files in os.walk(os.path.join(directory, domain)):
        if files:
            # Filter files based on keywords; the '_' after the variable name ensures that we don't grab the bias corrected or filtered datasets
            files = [file for file in files if ('r1i1p1' in file and 'day' in file and ('tasmin_' in file or 'tasmax_' in file or 'pr_' in file))]
            if not files:
                continue
            # Extract start and end dates from filenames
            start_date, end_date = extract_dates_from_filenames(files)
            
            # Remove dates from file name
            # Find the length of the substring to remove
            substring_length = len("_19500101-20051230.nc")

            # Remove the substring using string splicing
            new_filename = files[0][:-substring_length] + files[0][:0]
            
            # Generate output file path
            input_file = os.path.join(root, f'{new_filename}_*.nc')
            output_file = os.path.join(output_path, domain, f'{new_filename}_{start_date}-{end_date}.nc')
            
            # Check if file already exists
            if os.path.exists(output_file):
                
                print("File already exists: " + output_file)
            
            else:
                
                # Merge files using CDO command
                merge_files(input_file, output_file)

# Call the function to start parsing the master directory
# DOMAINS in all-caps to prevent accidental running
for domain in DOMAINS:
    parse_directory(master_path, domain)

## Step 2: Interpolate and remap the AUS-22i datasets to AUS-44i (0.5 X 0.5 deg)
Note, bilinear interpolation will be used for temperature data and first order conservative regridding will be used for precipitation data <br>
The 'my_projection_AUS22.txt' file was created prior to running the section below using the CDO command: <br>
cdo griddes tasmin_AUS-22_NCC-NorESM1-M_rcp85_r1i1p1_ICTP-RegCM4-7_v0_day_20060101-20991231.nc > my_projection_AUS22.txt <br>
This step assumes a consistent grid description for files of the same domain. <br>
Also note, the AUS-44 files had previously been interpolated to the AUS-44i grid.

In [90]:
# List regions (i.e. domains) that we are interested in and time periods for sorting
domains = ["AUS-22"]
time_periods = ["historical", "rcp45", "rcp85"]

# Define input and output directories
input_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step1_concat/'
output_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/'

In [ ]:
# Loop through directories 
# DOMAINS changed to all-caps to prevent accidental running of the script
for domain in DOMAINS:
    
    for time_period in time_periods:
        
        # Iterate through the files in the given directory
        for filename in os.listdir(input_master_path + domain + '/' + time_period):
            
            print("Starting: ", filename)
            
            # Define new filename after regridding and full output path
            new_filename = filename.replace(f"{domain}", "AUS-44i")
            new_full_output_path = os.path.join(output_master_path, time_period, new_filename)
            
            print("New output path : ", new_full_output_path)
            
            # Check if the file already exists
            if os.path.exists(new_full_output_path):
                print("File already exists: " + new_full_output_path)
                
            else:
                
                # Set the gridtype and store the changed gridtype as 'temp.nc'
                !cdo setgrid,{input_master_path + 'my_projection_AUS22.txt'} {input_master_path + domain + '/' + time_period + '/' + filename} {output_master_path + 'temp.nc'}
            
                # Perform the regridding using cdo remapycon (first order conservative regriddingand save in the folder based on the time period
                if filename.startswith('pr_AUS-22'):
                
                    !cdo remapycon,{output_master_path + 'AUS_44i_time1.nc'} {output_master_path + 'temp.nc'} {new_full_output_path}
            
                # perform bilinear interpolation for the temperature files
                elif filename.startswith('tasm'):
                
                    !cdo remapbil,{output_master_path + 'AUS_44i_time1.nc'} {output_master_path + 'temp.nc'} {new_full_output_path}

## Step 3: Create a new intake catalogue with necessary metadata for the interpolated files
This intake catalogue file needs to be updated in the preprocess.py script of the cordex-climpact software. 

In [25]:
# Read original catalogue
old_catalogue = pd.read_csv('/g/data/w40/ri9247/josh/catalogue_AUS44i.csv')

# Option to print original catalogue for sanity check
old_catalogue

In [27]:
# Define a new catalogue with the same columns as the original catalogue
new_catalogue = pd.DataFrame(columns = old_catalogue.columns)

In [28]:
# Define constants for new catalogue
project = 'cordex'
product = 'output'

# The version is retained in the file's metadata and gets dropped in Scott's code. For efficiency, we'll force a version, choosing the birthday shared by me and Baby Spice. It is also National Hug Day. 
version = 'v19920121'

# Define list with experiments
experiments = [
    'historical',
    'rcp85',
    'rcp45'
]

# Define master path of input datasets
master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/'

**NOTES:** <br>
Loop through the directory of input data and fill in the rest of the columns in the catalogue. <br>
Note that the files follow the following naming convention: <br>
{variable} _ {domain} _ {driving_model} _ {experiment} _ {ensemble} _ {institue}-{rcm_name} _ {rcm_version} _ {time_frequency} _ {date_range}.nc <br>
{variable} _ {domain} _ {driving_model} _ {experiment} _ {ensemble} _ {rcm_name_id} _ {rcm_version} _ {time_frequency} _ {date_range}.nc <br>
An example: <br>
tasmin _ AUS-44i _ NOAA-GFDL-GFDL-ESM2M _ rcp85 _ r1i1p1 _ CSIRO-CCAM-2008 _ v2 _ day _ 20060101-20991231.nc

In [29]:
# EXPERIMENTS in all caps to prevent accidental running
# Loop through the time period (aka experiments) directories in our pre-processed files
for experiment in EXPERIMENTS:
    directory = master_path + experiment
    
    # Get the individual files
    for filename in os.listdir(directory):
        
        # Extract the different attributes from the filename
        attributes = filename.split('_')
        
        # Define each attribute in the list (see above file naming structure
        variable = attributes[0]
        domain = attributes[1]
        driving_model = attributes[2]
        # experiment is defined via the looping
        ensemble = attributes[4]
        rcm_name_id = attributes[5]
        
        # A bit of hard-coding because the institute name for 'CLMcom-HZG' has a hyphen in it
        if 'CLMcom-HZG' in rcm_name_id:
            institute = 'CLMcom-HZG'
            rcm_name = 'CCLM5-0-15'
              
        else:
            # split the rcm_name_id at the first hyphen to get the institute and the rcm_name
            rcm_attributes = rcm_name_id.split('-', 1)
            institute = rcm_attributes[0].strip()
            rcm_name = rcm_attributes[1].strip()
            
        rcm_version = attributes[6]
        time_frequency = attributes[7]
        date_range = attributes[8]
        
        # Define the full path
        path = master_path + experiment + '/' + filename
        
        # Make a new dataframe with the file entry
        catalogue_entry = pd.DataFrame({'rcm_name': rcm_name,
                                        'driving_model': driving_model,
                                        'experiment': experiment,
                                        'path': path,
                                        'project': project,
                                        'product': product,
                                        'domain': domain,
                                        'institute': institute,
                                        'ensemble': ensemble,
                                        'rcm_name_id': rcm_name_id,
                                        'rcm_version': rcm_version,
                                        'time_frequency': time_frequency,
                                        'variable': variable,
                                        'version': version,
                                        'date_range': date_range
                                       }, index=[0])
        
        # Add everything to the catalogue
        new_catalogue = pd.concat([new_catalogue, catalogue_entry], ignore_index = True)
        

In [ ]:
# Option to print new_catalogue to check
pd.set_option('display.max_colwidth', None)
new_catalogue

In [31]:
# Write new catalogue to a csv file
new_catalogue.to_csv('/g/data/w40/ri9247/josh/catalogue.csv',index = False)

## Step 4 - Troubleshooting
Here, we troubleshoot a few simulations with unique pre-processing issues including:
- subsetting historical and RCP85 WRF pr files to match the latitude and longitude of the tasmin/tasmax
- correcting CCAM-2008 calendar mismatch
- NOAA-GFDL-GFDL-ESM2M CCAM-2008 troubleshooting: fill February 2003 with missing data values

### Subsetting WRF pr files

In [6]:
# Define list with experiments
experiments = [
    'historical',
    'rcp85'
]

# Define master path of input datasets
input_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/tmp_WRF_pr_files/'
output_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/'

In [ ]:
# EXPERIMENTS in all caps to prevent accidental running
# Loop through the time period (aka experiments) directories in our pre-processed files
for experiment in EXPERIMENTS:
        
    # Iterate through the files in the given directory
    for filename in os.listdir(input_master_path + experiment):
            
        print("Starting: ", filename)
        
        # Use CDO to subset lat and lon boundaries
        !cdo -sellonlatbox,89.75,205.75,9.75,-51.75 {input_master_path + experiment + '/' + filename} {output_master_path + experiment + '/' + filename}

### Correcting the calendar for CCAM-2008 files

In [4]:
# Define list with experiments
experiments = [
    'historical',
    'rcp45',
    'rcp85'
]

# Define master path of input datasets
input_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/tmp_CCAM_calendar/'
output_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/'

In [ ]:
# EXPERIMENTS in all caps to prevent accidental running
# Loop through the time period (aka experiments) directories in our pre-processed files
for experiment in EXPERIMENTS:
        
    # Iterate through the files in the given directory
    for filename in os.listdir(input_master_path + experiment):
        
        print("Starting: ", filename)
        
        # Use CDO to update the calendar
        # Make the unit of equal time
        !cdo -setreftime,1900-01-01,00:00:00,1day {input_master_path + experiment + '/' + filename} {'temp_cal_step1.nc'}
        
        # Make all files share the 365-day calendar
        !cdo -setcalendar,365_day {'temp_cal_step1.nc'} {output_master_path + experiment + '/' + filename}

### NOAA-GFDL-GFDL-ESM2M CCAM-2008 fill February 2003 with missing data values

In [8]:
# Define path to data input and output
master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/tmp_CCAM_missing_Feb/'

# Files we need to fill with missing values for February 2003
input_pr = 'pr_AUS-44i_NOAA-GFDL-GFDL-ESM2M_historical_r1i1p1_CSIRO-CCAM-2008_v1_day_19600101-20051231.nc'
input_tasmin = 'tasmin_AUS-44i_NOAA-GFDL-GFDL-ESM2M_historical_r1i1p1_CSIRO-CCAM-2008_v1_day_19600101-20051231.nc'
input_tasmax = 'tasmax_AUS-44i_NOAA-GFDL-GFDL-ESM2M_historical_r1i1p1_CSIRO-CCAM-2008_v1_day_19600101-20051231.nc'

In [9]:
# Load existing netCDF data
existing_pr = xr.open_dataset(master_path + input_pr)
existing_tasmin = xr.open_dataset(master_path + input_tasmin)
existing_tasmax = xr.open_dataset(master_path + input_tasmax)

# Define the time range we need to fill
start_date = np.datetime64('2003-02-01')
end_date = np.datetime64('2003-02-28')

# Create a new time axis with the needed time range
new_time = np.arange(start_date, end_date + np.timedelta64(1, 'D'), np.timedelta64(1, 'D'))

In [2]:
# Define missing value based on value in file's metadata
missing_value_fill = 1.e+20

In [11]:
# Create new datasets with the copied metadata
new_pr = xr.Dataset(attrs=existing_pr.attrs)
new_pr['time'] = new_time

new_tasmin = xr.Dataset(attrs=existing_tasmin.attrs)
new_tasmin['time'] = new_time

new_tasmax = xr.Dataset(attrs=existing_tasmax.attrs)
new_tasmax['time'] = new_time

In [12]:
# Create a new DataArray with missing values
# Precipitation
missing_data_pr = np.full((len(new_time), len(existing_pr['lat']), len(existing_pr['lon'])), missing_value_fill)
new_pr['pr'] = xr.DataArray(missing_data_pr,
                                  coords={'time': new_time, 'lat': existing_pr['lat'], 'lon': existing_pr['lon']},
                                  dims=['time', 'lat', 'lon'])

# Tasmin
missing_data_tasmin = np.full((len(new_time), len(existing_tasmin['lat']), len(existing_tasmin['lon'])), missing_value_fill)
new_tasmin['tasmin'] = xr.DataArray(missing_data_tasmin,
                                  coords={'time': new_time, 'lat': existing_tasmin['lat'], 'lon': existing_tasmin['lon']},
                                  dims=['time', 'lat', 'lon'])

# Tasmax
missing_data_tasmax = np.full((len(new_time), len(existing_tasmax['lat']), len(existing_tasmax['lon'])), missing_value_fill)
new_tasmax['tasmax'] = xr.DataArray(missing_data_tasmax,
                                  coords={'time': new_time, 'lat': existing_tasmax['lat'], 'lon': existing_tasmax['lon']},
                                  dims=['time', 'lat', 'lon'])

In [13]:
# Write missing data files to new datasets
new_pr.to_netcdf(master_path + 'pr_Feb2003_missing_fill.nc')
new_tasmin.to_netcdf(master_path + 'tasmin_Feb2003_missing_fill.nc')
new_tasmax.to_netcdf(master_path + 'tasmax_Feb2003_missing_fill.nc')

### Convert calendar types to a consistent type.

In [14]:
master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/tmp_CCAM_missing_Feb/'

In [ ]:
# Iterate through the files in the given directory
for filename in os.listdir(master_path):
        
    print("Starting: ", filename)
        
    # Use CDO to update the calendar
    # Make the unit of equal time
    !cdo -setreftime,1900-01-01,00:00:00,1day {master_path + filename} {master_path + 'temp1_' + filename}
        
    # Make all files share the 365-day calendar
    !cdo -setcalendar,365_day {master_path + 'temp1_' + filename} {master_path + 'step2_' + filename}

**NOTES:** <br>
Used cdo mergetime in the terminal to merge the step2_{variable}_*.nc files <br>
Useful scripts below to check that filling worked correctly

In [ ]:
# Check for correct number of days in each year
f_pr = xr.open_dataset('/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/tmp_CCAM_missing_Feb/filled_tasmin_AUS-44i_NOAA-GFDL-GFDL-ESM2M_historical_r1i1p1_CSIRO-CCAM-2008_v1_day_19600101-20051231.nc')
for y in range(1960,2006):
    ts_pr = f_pr.time.sel(time=slice(str(y)+'-01',str(y)+'-12'))
    print(str(y)+': '+str(len(ts_pr)))

In [ ]:
# Check that the months have the correct number of days
for m in range(1,13):
    month = str(m).zfill(2)
    ts_pr = f_pr.time.sel(time=slice('2003-'+month,'2003-'+month))
    print(month+': '+str(len(ts_pr)))

## Copy indices into folders based on the experiment and index name
Note, ahead of doing this, the empty folders were copied from an existing directory using: <br>
find . -maxdepth 1 -type d > /scratch/w40/ri9247/tmp/dirs.txt <br>
xargs mkdir -p < /scratch/w40/ri9247/tmp/dirs.txt 

In [4]:
# Original output directory of Climpact indices
root_dir_climpact = '/g/data/w40/yl1269/climpact_indices/cordex_AUS44i_Aug2023_all_indices/'

# Master destination directory
dest_root_dir = '/scratch/w40/ri9247/tmp/cordex_indices/'

# List of filenames created that are not climate indices that we will not sort
non_index_climpact_filenames = ['thresholds.done'
    , 'thresholds.log'
    , 'thresholds.nc'
    , 'climpact.done'
    , 'climpact.log']

In [ ]:
# Loop through the original climpact output directory (files are sorted based on the simulation and the experiment)
for model_dir in os.listdir(root_dir_climpact):
    
    # Print directory name for tracking progress
    print(model_dir)
    
    # Get the experiment based on the original output directory name
    if 'historical' in model_dir:
        experiment = 'historical/'
        
    elif 'rcp45' in model_dir: 
        experiment = 'rcp45/'
        
    elif 'rcp85' in model_dir: 
        experiment = 'rcp85/'
    
    # For each directory loop through the files and copy the indices files into the appropriate folder
    for file in os.listdir(root_dir_climpact + model_dir):
        
        # If the file is not a climate index file, skip it
        if file in non_index_climpact_filenames: continue
        
        # Get the file info
        attributes = file.split("_")
        
        # Get index
        index = attributes[0] + '/'
        
        # Define the new directory
        new_directory = dest_root_dir + experiment + index
        
        # Copy the file to the new, sorted directory
        !cp {root_dir_climpact + model_dir + '/' + file} {new_directory + file}